# Setup

## Installations and imports

### Installations
As some libraries that are not in the default version in colab are used, it is necessary to install them

In [1]:
!pip install torch-scatter -f https://pytorch-geometric.com/whl/torch-1.9.0+cu102.html
!pip install torch-sparse -f https://pytorch-geometric.com/whl/torch-1.9.0+cu102.html
!pip install torch-cluster -f https://pytorch-geometric.com/whl/torch-1.9.0+cu102.html
!pip install torch-geometric -f https://pytorch-geometric.com/whl/torch-1.9.0+cu102.html

!pip install torchmetrics

Looking in links: https://pytorch-geometric.com/whl/torch-1.9.0+cu102.html
Looking in links: https://pytorch-geometric.com/whl/torch-1.9.0+cu102.html
Looking in links: https://pytorch-geometric.com/whl/torch-1.9.0+cu102.html
Looking in links: https://pytorch-geometric.com/whl/torch-1.9.0+cu102.html


### Imports
In the next snippet of code there are all the imports necessaries for the project and the tensorboard is initialized.

In [2]:
import plotly.graph_objects as go

import torch
from torch import nn
from torch.nn import functional as F

from torchmetrics import Accuracy

from torch_geometric.data import DataLoader
from torch_geometric.utils import to_dense_batch
from torch_geometric.datasets import ModelNet
from torch_geometric.transforms import SamplePoints, NormalizeScale

## Hyperparameters

In [3]:
hparams = {
    'bs': 1,
    'device': torch.device('cuda:0' if torch.cuda.is_available() else 'cpu'),
    'drive_root': '/content/drive/MyDrive/Dataset/ModelNet',
    'normalize_scale': True, 
    'fixed_num_of_points': 1024,
    'model_log': '/content/drive/MyDrive/Adapt/pointNet_normalized_flip_rotation_0.3_Adam_OneCycleLR.pt', 
    'k': 3,
    'num_classes': 10,
    'dropout': 0.3
}

#Model

In [4]:
class TNet(nn.Module):
  def __init__(self, k=3):
    super().__init__()
    self.k = k
    
    self.Conv1 = nn.Conv1d(in_channels=k, out_channels=64, kernel_size=1)
    self.bn1 = nn.BatchNorm1d(64) 
    self.Conv2 = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=1)
    self.bn2 = nn.BatchNorm1d(128)
    self.Conv3 = nn.Conv1d(in_channels=128, out_channels=1024, kernel_size=1)
    self.bn3 = nn.BatchNorm1d(1024)

    self.FC1 = nn.Linear(in_features=1024, out_features=512)
    self.bn4 = nn.BatchNorm1d(512)
    self.FC2 = nn.Linear(in_features=512, out_features=256)
    self.bn5 = nn.BatchNorm1d(256)

    self.FC3 = nn.Linear(in_features=256, out_features=k*k)

  def forward(self, cloud_points):
    bs = cloud_points.size(0)

    x = F.relu(self.bn1(self.Conv1(cloud_points)))
    x = F.relu(self.bn2(self.Conv2(x)))
    x = F.relu(self.bn3(self.Conv3(x)))

    # size: [batch size, 1024, # of points]
    x = nn.MaxPool1d(x.size(-1))(x) # pool with kernel = # of points/batch
    # size: [batch size, 1024, 1]
    x = x.view(bs,-1) # flatten to get horizontal vector

    # size: [batch size, 1024]
    x = F.relu(self.bn4(self.FC1(x)))
    x = F.relu(self.bn5(self.FC2(x)))

    # diagonal matrices initialized, as many as batch size
    init_matrix = torch.eye(self.k, requires_grad=True).repeat(bs,1,1)
    if x.is_cuda:
      init_matrix = init_matrix.cuda() # gets updated according to f.c. output
    matrix = self.FC3(x).view(-1, self.k, self.k) + init_matrix
    return matrix

class Transform(nn.Module):
   def __init__(self, k=3):
        super().__init__()
        self.input_transform = TNet(k)
        self.feature_transform = TNet(k=64)

        self.Conv1 = nn.Conv1d(in_channels=3, out_channels=64, kernel_size=1)
        self.Conv2 = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=1)
        self.Conv3 = nn.Conv1d(in_channels=128, out_channels=1024, kernel_size=1)

        self.bn1 = nn.BatchNorm1d(64)
        self.bn2 = nn.BatchNorm1d(128)
        self.bn3 = nn.BatchNorm1d(1024)

   def forward(self, x):
        bs = x.size(0)

        matrix3x3 = self.input_transform(x)
        x = torch.bmm(torch.transpose(x,1,2),matrix3x3).transpose(1,2)
        x = F.relu(self.bn1(self.Conv1(x)))
        
        matrix64x64 = self.feature_transform(x)
        x = torch.bmm(torch.transpose(x,1,2), matrix64x64).transpose(1,2) 
        x = F.relu(self.bn2(self.Conv2(x)))
        x = self.bn3(self.Conv3(x))

        x = nn.MaxPool1d(x.size(-1))(x)
        global_features = x.view(bs,-1)
        return global_features

class PointNetModel(nn.Module):
    def __init__(self, k=3, num_classes=16, dropout=0.3):
        super().__init__()
        self.transform = Transform(k)

        self.FC1 = nn.Linear(in_features=1024, out_features=512)
        self.bn1 = nn.BatchNorm1d(512)
        self.FC2 = nn.Linear(in_features=512, out_features=256)
        self.bn2 = nn.BatchNorm1d(256)
        self.FC3 = nn.Linear(in_features=256, out_features=num_classes)
        self.dropout = nn.Dropout(p=0.3)

    def forward(self, x):
        global_features = self.transform(x)
        x = F.relu(self.bn1(self.FC1(global_features)))
        x = self.FC2(x)
        # apply dropout if exists
        x = self.dropout(x) if self.dropout is not None else x
        x = F.relu(self.bn2(x))
        output = self.FC3(x)
        return output, F.softmax(output,dim=1)

# Dataset
First of all, it is necessary to make the drive folder with the dataset available to this collab in order not to download it every time. 

In [5]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


## Transformations
In this project is necessary to use some transformations to either normalize the data or to perform data augmentation.

In [6]:
def get_pre_transformation(number_points=1024):
    return SamplePoints(num=number_points)

def get_transformation(normalize_scale):
    if normalize_scale:  
      return NormalizeScale()
    else:
        return None

## Test data

In [7]:
def get_dataset(root, number_points=1024, normalize_scale=True):
    
    test_dataset = ModelNet(root=root, name="10", train=False, pre_transform=get_pre_transformation(number_points), transform=get_transformation(normalize_scale))
    return test_dataset

## Helper functions
As there are some functionalities that are used by different functions or can be used in the future, a list of helpers fucntions has been created

In [8]:
# Method to visualize a cloud point
def visualize_point_cloud(point_cloud):
    points, y = point_cloud
    fig = go.Figure(data=[go.Mesh3d(x=points[1][:, 0], y=points[1][:, 1], z=points[1][:, 2], mode='markers', marker=dict(size=3, opacity=1))])
    fig.show()

# Testing inference

In [9]:
def test(test_data, model_state_dict_root):
    test_loader = DataLoader(test_data, batch_size=hparams['bs'], shuffle=False)
    model = PointNetModel(hparams['k'], hparams['num_classes'], hparams['dropout']).to(hparams['device'])

    accuracy = Accuracy(average='micro', compute_on_step=False).to(hparams['device'])
    model.load_state_dict(torch.load(model_state_dict_root))
    model.eval()
    # Metric stored information reset:
    accuracy.reset()
    # Batch loop for validation:
    with torch.no_grad():
        for i, data in enumerate(test_loader, 1):
            # Data retrieval from each bath:
            points = to_dense_batch(data.pos, batch=data.batch)[0].to(hparams['device']).float().transpose(1, 2)
            targets = data.y.to(hparams['device'])

            # Forward pass:
            preds, probs = model(points)

            # Batch metrics calculation:
            accuracy.update(probs, targets)

    mean_accu = accuracy.compute().item()
    # Print of all metrics:
    print("Test Acc.: ", mean_accu)
    return mean_accu

In [10]:
test_dataset = get_dataset(hparams['drive_root'], hparams['fixed_num_of_points'], hparams['normalize_scale'])
test(test_dataset, hparams['model_log']) 

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:652: UserWarning:

Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)



Test Acc.:  0.9240087866783142


0.9240087866783142